In [1]:
import os
os.chdir('/Users/nicolasmartinet/Desktop/election')

print(os.getcwd())


/Users/nicolasmartinet/Desktop/election


In [2]:
import urllib.request
import json

url = 'https://apisds.paddypower.com/sdspp/competition-page/v3?_ak=vsd0Rm5ph2sS2uaK&betexRegion=IRL&capiJurisdiction=intl&competitionId=12306853&countryCode=IE&currencyCode=EUR&eventTypeId=2378961&exchangeLocale=en_GB&includeBadges=true&includeLayout=true&includePrices=true&includeSeoCards=true&includeSeoFooter=true&language=en&loggedIn=false&regionCode=IRE'
req = urllib.request.Request(url)
req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
req.add_header('Accept-Language', 'en-US,en;q=0.5')

r = urllib.request.urlopen(req).read().decode('utf-8')
response_json = json.loads(r)


In [3]:
def get_runners(obj):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == "runners":
                yield value
            else:
                yield from get_runners(value)
    elif isinstance(obj, list):
        for item in obj:
            yield from get_runners(item)


In [4]:
runners = []
for runners_list in get_runners(response_json):
    runners.extend(runners_list)


In [5]:
odds_ratios = []
for runner in runners:
    runner_name = runner["runnerName"]
    numerator = runner["winRunnerOdds"]["trueOdds"]["fractionalOdds"]["numerator"]
    denominator = runner["winRunnerOdds"]["trueOdds"]["fractionalOdds"]["denominator"]
    odds_ratio = numerator / denominator
    odds_ratios.append((runner_name, odds_ratio))


In [6]:
sorted_odds_ratios = sorted(odds_ratios, key=lambda x: x[1])


In [7]:
def find_nominee_market(obj, marketName):
    if isinstance(obj, dict):
        if "marketName" in obj and obj["marketName"] == marketName:
            return obj
        for key, value in obj.items():
            result = find_nominee_market(value, marketName)
            if result is not None:
                return result
    elif isinstance(obj, list):
        for item in obj:
            result = find_nominee_market(item, marketName)
            if result is not None:
                return result
    return None


In [8]:
Election_Winner = find_nominee_market(response_json, "Election Winner")
Republican_nominee = find_nominee_market(response_json, "Republican Nominee")
Democrat_nominee = find_nominee_market(response_json, "Democratic Nominee")

In [9]:
import pandas as pd

def create_runners_dataframe(d):
    runners = d.get("runners", [])
    data = []
    for runner in runners:
        selection_id = runner.get("selectionId")
        handicap = runner.get("handicap")
        runner_name = runner.get("runnerName")
        sort_priority = runner.get("sortPriority")
        runner_status = runner.get("runnerStatus")
        decimal_odds = runner.get("winRunnerOdds", {}).get("trueOdds", {}).get("decimalOdds", {}).get("decimalOdds")
        numerator = runner.get("winRunnerOdds", {}).get("trueOdds", {}).get("fractionalOdds", {}).get("numerator")
        denominator = runner.get("winRunnerOdds", {}).get("trueOdds", {}).get("fractionalOdds", {}).get("denominator")
        probability = 1 / decimal_odds * 100
        data.append((selection_id, handicap, runner_name, sort_priority, runner_status, decimal_odds, numerator, denominator, probability))
    df = pd.DataFrame(data, columns=["selectionId", "handicap", "runnerName", "sortPriority", "runnerStatus", "decimalOdds", "numerator", "denominator", "probability"])
    df.probability = df.probability/df.probability.sum()
    df = df.sort_values(by="decimalOdds")
    return df


In [10]:
create_runners_dataframe(Election_Winner).head()

,selectionId,handicap,runnerName,sortPriority,runnerStatus,decimalOdds,numerator,denominator,probability
163,6816445,0,Joe Biden,164,ACTIVE,2.625,13,8,0.175437
53,10874213,0,Donald Trump,54,ACTIVE,3.250,9,4,0.141699
66,30371713,0,Ron DeSantis,67,ACTIVE,6.000,5,1,0.076754
2,53349422,0,Robert F.Kennedy Jr,3,ACTIVE,21.000,20,1,0.021930
133,12126964,0,Kamala Harris,134,ACTIVE,26.000,25,1,0.017712


In [11]:
def save(*dfs, names):
    import datetime
    today = datetime.date.today()
    for df, name in zip(dfs, names):
        filename = f"{name}_{today.day}-{today.month}-{today.year}.csv"
        df.to_csv(filename, index=False)

In [12]:
save(create_runners_dataframe(Election_Winner), create_runners_dataframe(Republican_nominee), create_runners_dataframe(Democrat_nominee), names = ["Election_Winner","Republican_nominee", "Democrat_nominee" ])